# IPython Parallel Computing Guide

This guide is intended for IPython 4.0 or greater and Jupyter notebooks. 
Make sure you install `ipyparallel` using `conda install ipyparallel`.
Throughout this guide I assume you are wanting to connect to an IPython cluster
via a Jupyter notebook. I think it's possible to connect via a command line
IPython interactive session as well.

This guide will help you get an IPython cluster up and running on our cluster.
To learn the different ways of running code etc. on an IPython cluster, you 
should consult the [`ipyparallel` documentation](https://ipyparallel.readthedocs.org/en/latest/index.html). 
[This page](https://ipyparallel.readthedocs.org/en/latest/multiengine.html) is a 
good starting place after you've gone through this notebook.

## Local Parallel Computing

This first part is based on [this page](https://ipyparallel.readthedocs.org/en/latest/intro.html#getting-started).

The simplest way to do execute code in parallel is by starting an IPython cluster on 
the local machine. For instance, if you are logged in on a head node, you can start an 
IPython cluster with four engines (cores) by executing 

    ipcluster start -n 4
    
at the command line. You should run this in a screen because you will need this
command to continue running as long as you want to use the IPython cluster. You 
could also run this command in the background (writing the logs to a file I guess).

Note that you probably want to have the same `conda` environment loaded when you launch the 
cluster that you are using to run this notebook. Using different environments may not
cause a problem although it would be confusing because the engines' Python environment
would be different than the environment of this notebook.
    
Now that we've started the cluster, we can import `ipyparallel` in this notebook and create a client. The
client automatically looks for the cluster and connects to it. If you are using
an IPython profile besides the default, you may need `c = Client(profile='myprofile')`.

In [2]:
from ipyparallel import Client
# Connect to client.
c = Client()
# You may need to specify the profile: c = Client(profile='myprofile')

# Show the IDs of the cluster engines:
c.ids

In [4]:
# Run some code on each cluster engine.
c[:].apply_sync(lambda : "Hello, World")

['Hello, World', 'Hello, World', 'Hello, World', 'Hello, World']

Hello, World prints four times because the code was executed on each one
of our four engines.

### Stopping the cluster

You should be able to use

    ipcluster stop
    
or 

    ipcluster stop --profile myprofile
    
at the command line to stop the IPython cluster. You can also kill the process
wherever it's running, via top, etc.

## SGE Parallel Computing

Now we'll look at controlling the IPython cluster from the head node but executing code in parallel 
on the scheduled nodes **and/or** on the head node as well. 
This second part is based on [this page](https://ipyparallel.readthedocs.org/en/latest/process.html)
although the page contains a lot of stuff not relevant to our cluster set up.

When running an IPython cluster on a local machine as we did above, you can start the 
cluster with `ipcluster` which launches both a controller and engines. The controller
coordinates the task of sending code out to the engines while the engines actually do the
work. Here, we want to run a controller on the head node and engines out on the scheduled 
nodes (and potentially on the head node as well). This will allow us to farm out parallel
computations from notebooks running on the headnode.

An alternative strategy could be to
configure the IPython cluster to use SGE and use `ipcluster` to launch an entire cluster
(controller and engines) on a scheduled node. In this scenario, you would need to SSH 
to that node and launch a notebook to connect to the cluster. This is a bit harder since
we don't have web servers running on the scheduled nodes, so I guess you would need to SSH 
tunnel to those notebook to see them. If you look online you will probably see people 
using this strategy because this is what you would do on a cluster/cloud system like SDSC
or Amazon.

### Set up

This set up only needs to be done once to get an IPython profile with some parameters
particular to our cluster. First we'll create an IPython profile with parallel config files:

    ipython profile create parallel --parallel

Change to `~/.ipython/profile_parallel`. We want our controller to listen for 
external connections. Because we are on the head node, we are on a trusted network, 
so we can listen for all external connections. Open `ipcontroller_config.py` and 
uncomment/change the following line to allow all external connections:

    c.HubFactory.engine_ip = '*'
    
Also add the line

    c.HubFactory.ip = '*'
    
right below this.

### Conda environment

Note that you probably want to use the same `conda` environment that this notebook is running
under for all of the steps below. 
Using different environments may not cause a problem although it would definitely be confusing.

### Launch controller

You can launch the controller on the head node using 

    ipcontroller --profile=parallel
    
while logged into the head node. Run this in a screen so it can in the background (or run
in the background and write the logs to files). This controller
isn't doing anything except waiting for engines so that it can manage them. You should
see some output like

    2015-12-02 16:29:16.970 [IPControllerApp] Hub listening on tcp://*:48181 for registration.
    2015-12-02 16:29:16.971 [IPControllerApp] Hub using DB backend: 'NoDB'
    2015-12-02 16:29:17.244 [IPControllerApp] hub::created hub
    2015-12-02 16:29:17.244 [IPControllerApp] writing connection info to /frazer01/home/cdeboever/.ipython/profile_parallel/security/ipcontroller-client.json
    2015-12-02 16:29:17.248 [IPControllerApp] writing connection info to /frazer01/home/cdeboever/.ipython/profile_parallel/security/ipcontroller-engine.json
    2015-12-02 16:29:17.250 [IPControllerApp] task::using Python leastload Task scheduler
    2015-12-02 16:29:17.251 [IPControllerApp] Heartmonitor started
    2015-12-02 16:29:17.263 [IPControllerApp] Creating pid file: /frazer01/home/cdeboever/.ipython/profile_parallel/pid/ipcontroller.pid
    2015-12-02 16:29:17.273 [scheduler] Scheduler started [leastload]
    2015-12-02 16:29:17.275 [IPControllerApp] client::client '\x00o\xee1b' requested u'connection_request'
    2015-12-02 16:29:17.275 [IPControllerApp] client::client ['\x00o\xee1b'] connected

### Launch engines on the head node

First, let's try launching some engines on the head node. You can launch an engine
on the head node using

    ipengine --profile=parallel
    
You should see some output like

    2015-12-02 16:30:03.773 [IPEngineApp] Loading url_file u'/frazer01/home/cdeboever/.ipython/profile_parallel/security/ipcontroller-engine.json'
    2015-12-02 16:30:03.904 [IPEngineApp] Registering with controller at tcp://127.0.0.1:48181
    2015-12-02 16:30:03.974 [IPEngineApp] Starting to monitor the heartbeat signal from the hub every 3010 ms.
    2015-12-02 16:30:03.978 [IPEngineApp] Completed registration with id 0
    
The `ipcontroller-engine.json` file was created when we launched our controller and tells
the engines where to find the controller so they can connect. If an engine can't find a controller,
it will die. You can see from this output that our engine connected to the controller correctly.
Now we can start a client in this notebook that will connect to our controller.

In [3]:
from ipyparallel import Client
# Connect to client.
c = Client(profile='parallel')

# Show the IDs of the cluster engines:
c.ids

[0]

You can see that right now we just have a single engine. Go ahead and start 
another engine on the head node using the same command as before:

    ipengine --profile=parallel
    
We'll make a new client and now we can see that both engines are connected to the 
controller.

In [4]:
c = Client(profile='parallel')

# Show the IDs of the cluster engines:
c.ids

[0, 1]

Now we have engines with IDs 0 and 1. If you look at the output when you 
started the last engine, you'll see something like

    2015-12-02 16:30:53.280 [IPEngineApp] Loading url_file u'/frazer01/home/cdeboever/.ipython/profile_parallel/security/ipcontroller-engine.json'
    2015-12-02 16:30:53.301 [IPEngineApp] Registering with controller at tcp://127.0.0.1:48181
    2015-12-02 16:30:53.371 [IPEngineApp] Starting to monitor the heartbeat signal from the hub every 3010 ms.
    2015-12-02 16:30:53.375 [IPEngineApp] Completed registration with id 1
    
Notice that the last line says that we just registered a new engine with id 1 which corresponds
to what we see from `c.ids`.

As before, we can run some code in parallel on both engines:

In [5]:
# Run some code on each cluster engine.
c[:].apply_sync(lambda : "Hello, World")

['Hello, World', 'Hello, World']

### Launch engines on the scheduled nodes

Now let's launch some engines on the scheduled nodes. 
You need to make an SGE script to launch engines on the scheduled nodes. Try making a script
`~/test.sh` with the following contents:

Note that you may want/need to activate a particular `conda` environment in this script
if you are using an environment besides the default for this notebook/the controller.
You can submit the SGE scrip the schedule using `qsub ~/test.sh`. After a few moments, 
check `~/test.err`. You should see some output like:

    2015-12-02 16:32:56.771 [IPEngineApp] Loading url_file u'/frazer01/home/cdeboever/.ipython/profile_parallel/security/ipcontroller-engine.json'
    2015-12-02 16:32:57.199 [IPEngineApp] Registering with controller at tcp://169.228.63.175:48181
    2015-12-02 16:33:07.101 [IPEngineApp] Starting to monitor the heartbeat signal from the hub every 3010 ms.
    2015-12-02 16:33:07.212 [IPEngineApp] Completed registration with id 2
    
It looks like our engine connected successfully. You can see the job using `qstat':

Let's try creating a new client.

In [6]:
from ipyparallel import Client
# Connect to client.
c = Client(profile='parallel')

# Show the IDs of the cluster engines:
c.ids

[0, 1, 2]

Now we have three engines available:

In [7]:
# Run some code on each cluster engine.
c[:].apply_sync(lambda : "Hello, World")

['Hello, World', 'Hello, World', 'Hello, World']

As before, we can run code on all three. If we kill the scheduled job (using `qdel 7137`
in this case) that is running one of our engines, we'll get a problem:

In [8]:
c[:].apply_sync(lambda : "Hello, World")

CompositeError: one or more exceptions from call to method: <lambda>
[Engine Exception]EngineError: Engine 2 died while running task '893c9fad-266d-4659-b7b8-62b625aaf94d'

If we look at the logs from the controller, we'll see something like

    2015-12-02 15:42:34.660 [IPControllerApp] heartbeat::missed 6dbc2571-3838-47dc-95f8-8f56c1a22bab : 3
    2015-12-02 15:42:37.662 [IPControllerApp] heartbeat::missed 6dbc2571-3838-47dc-95f8-8f56c1a22bab : 4
    2015-12-02 15:42:40.663 [IPControllerApp] heartbeat::missed 6dbc2571-3838-47dc-95f8-8f56c1a22bab : 5
    2015-12-02 15:42:43.663 [IPControllerApp] heartbeat::missed 6dbc2571-3838-47dc-95f8-8f56c1a22bab : 6
    2015-12-02 15:42:46.663 [IPControllerApp] heartbeat::missed 6dbc2571-3838-47dc-95f8-8f56c1a22bab : 7
    2015-12-02 15:42:49.662 [IPControllerApp] heartbeat::missed 6dbc2571-3838-47dc-95f8-8f56c1a22bab : 8
    2015-12-02 15:42:52.663 [IPControllerApp] heartbeat::missed 6dbc2571-3838-47dc-95f8-8f56c1a22bab : 9
    2015-12-02 15:42:55.663 [IPControllerApp] heartbeat::missed 6dbc2571-3838-47dc-95f8-8f56c1a22bab : 10
    2015-12-02 15:42:58.663 [IPControllerApp] heartbeat::missed 6dbc2571-3838-47dc-95f8-8f56c1a22bab : 11
    2015-12-02 15:42:58.663 [IPControllerApp] registration::unregister_engine(2)
    
I think the `heartbeat::missed` lines are waiting for a reply from engine 2. Eventually, the
controller decides it won't hear back from engine 2 so it throws the above error and removes
engine 2:

In [9]:
c.ids

[0, 1]

I believe that if you kill an engine, the controller will eventually remove it after 
it doesn't hear from the engine for a set amount of heartbeats.

Note that if we submit a job to run an engine, it will run until it is killed by 
queue time limits. If you wanted to limit the time, you could do something like

This job would finish after an hour and kill the engine.

### Launch multiple engines on the scheduled nodes

Now let's try to launch multiple engines on the scheduled nodes. A simple way to do this
is to create a file `~/test2.sh` with the following contents:

Submit this to the scheduler with `qsub` and after a bit of time you should see 
output in `~/test2.err` like

    2015-12-02 16:52:22.952 [IPEngineApp] Loading url_file u'/frazer01/home/cdeboever/.ipython/profile_parallel/security/ipcontroller-engine.json
    '
    2015-12-02 16:52:22.952 [IPEngineApp] Loading url_file u'/frazer01/home/cdeboever/.ipython/profile_parallel/security/ipcontroller-engine.json
    '
    2015-12-02 16:52:22.984 [IPEngineApp] Registering with controller at tcp://169.228.63.175:48181
    2015-12-02 16:52:22.984 [IPEngineApp] Registering with controller at tcp://169.228.63.175:48181
    2015-12-02 16:52:23.100 [IPEngineApp] Starting to monitor the heartbeat signal from the hub every 3010 ms.
    2015-12-02 16:52:23.106 [IPEngineApp] Starting to monitor the heartbeat signal from the hub every 3010 ms.
    2015-12-02 16:52:23.107 [IPEngineApp] Completed registration with id 6
    2015-12-02 16:52:23.114 [IPEngineApp] Completed registration with id 5
    
We can create a new client and see these engines:

In [17]:
from ipyparallel import Client
# Connect to client.
c = Client(profile='parallel')

# Show the IDs of the cluster engines:
c.ids

[0, 1, 5, 6]

Note that my engines got IDs 5 and 6 because I've been experimenting with creating
engines and deleting them. It seems that IPython won't recycle engine numbers. Your
numbers may be different. Once again we can kill these engines by killing the job
using `qdel` or we can run the engines in the background and use `sleep` to kill the 
engines after a certain amount of time (see example above).

#### Array jobs

A more efficient way to launch multiple engines is to use 
[array jobs](https://wiki.duke.edu/display/SCSC/SGE+Array+Jobs). 
Create a file `~/test_array.sh` with

This script will launch four engines that will die after 5 minutes. They 
will look something like 

       7141 0.45617 engine_tes cdeboever    r     12/02/2015 17:07:49 all.q@fl-n-1-10                    1 1
       7141 0.45617 engine_tes cdeboever    r     12/02/2015 17:07:49 all.q@fl-n-1-10                    1 2
       7141 0.45617 engine_tes cdeboever    r     12/02/2015 17:07:49 all.q@fl-n-1-10                    1 3
       7141 0.45617 engine_tes cdeboever    r     12/02/2015 17:07:49 all.q@fl-n-1-10                    1 4
   
in `qstat`. We can create a new client and see these engines:

In [23]:
from ipyparallel import Client
# Connect to client.
c = Client(profile='parallel')

# Show the IDs of the cluster engines:
c.ids

[0, 1, 5, 6]

You can see from the output above that the engines all have the same job ID,
so you can easily kill them all with one `qdel`.

### Stopping the cluster

I'm not exactly sure what the best way to stop the cluster is, but killing the 
controller process (`ctrl-c` wherever it's running, use `top`, etc.) will stop the controller.
The engines will stop after failing to hear from the controller 50 times (engines
check for the controller ~3 seconds). You'll see something like

    2015-12-02 16:11:21.074 [IPEngineApp] WARNING | No heartbeat in the last 3010 ms (50 time(s) in a row).
    2015-12-02 16:11:21.075 [IPEngineApp] CRITICAL | Maximum number of heartbeats misses reached (50 times 3010 ms), shutting down.
    
from the engine logs.